
#   TEXT CLASSIFICATION USING ATTENTION MECHANISIM 


###  POC using Keras 

### Implementing attention mechanisim for  sentence-level sentiment analysis dataset collected from the University of California Irvine Machine Learning Repository

![alt text](Attention.png)

In [71]:
import csv
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
DIR_PATH = os.path.dirname(os.path.realpath('__file__'))
DATASETS = "datasets"
SENTIMENT_ANALYSIS = "sentiment labelled sentences"
file_path = os.path.join(DIR_PATH,DATASETS,SENTIMENT_ANALYSIS,"merged_dataset.txt")
df_sentiment = pd.read_csv(file_path, sep="\t", header=None)
df_sentiment.columns = ["review", "rating"]

In [21]:
df_sentiment.head()

,review,rating
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [22]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
review    2000 non-null object
rating    2000 non-null int64
dtypes: int64(1), object(1)
memory usage: 31.3+ KB


## Data preparation using keras preprocessing

In [74]:
t=Tokenizer()
t.fit_on_texts(df_sentiment["review"])
text_matrix=t.texts_to_sequences(df_sentiment["review"])

In [73]:
len_mat=[]
for i in range(len(text_matrix)):
    len_mat.append(len(text_matrix[i]))

In [92]:
label_encoder = LabelEncoder()
text_pad = pad_sequences(text_matrix, maxlen=32, padding='post')
integer_encode = label_encoder.fit_transform(df_sentiment["rating"])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encode.reshape(len(integer_encode), 1)
Y = onehot_encoder.fit_transform(integer_encoded)
Y.shape

/Users/deepanshu.kandpal/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(2000, 2)

## Model creation using Functional API of Keras

In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [89]:
inputs1= Input(shape=(32,))
x1=Embedding(input_dim=len(t.word_index.items())+1,output_dim=32,input_length=32,embeddings_regularizer=keras.regularizers.l2(.001))(inputs1)
x1=LSTM(100,dropout=0.3,recurrent_dropout=0.2)(x1)
outputs1=Dense(1,activation='sigmoid')(x1)
model1=Model(inputs1,outputs1)

In [90]:
model1.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 32, 32)            66304     
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 119,605
Trainable params: 119,605
Non-trainable params: 0
_________________________________________________________________


In [91]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model1.fit(x=text_pad,y=Y,batch_size=100,epochs=10,verbose=1,shuffle=True,validation_split=0.2)

ValueError: A target array with shape (2000, 2) was passed for an output of shape (None, 1) while using as loss `binary_crossentropy`. This loss expects targets to have the same shape as the output.